In [29]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from models.dit import DiT
from models import *

#### 테스트 데이터셋 클래스 정의

In [30]:
class TimeSeriesDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path, header=None)

        label_map = {label: idx for idx, label in enumerate(df.iloc[:, 0].unique())}
        df.iloc[:, 0] = df.iloc[:, 0].map(label_map)

        self.X = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32).unsqueeze(1)
        self.y = torch.tensor(df.iloc[:, 0].values, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

#### 모델 불러오기 및 평가 함수 정의

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test_dataset = TimeSeriesDataset("C:/Users/Pro/Desktop/AnomalyDiT-main/AnomalyDiT-main/Dataset/ECG_Test_with_anomaly.csv")
test_loader = DataLoader(test_dataset, batch_size=64)

model = DiT(input_size=750, patch_size=5, in_channels=1, num_classes=42).to(device)
model.load_state_dict(torch.load("dit_ts_model.pt"))
model.eval()

## 이전

In [5]:
train = pd.read_csv('C:/Users/Pro/Desktop/AnomalyDiT-main/AnomalyDiT-main/Dataset/ECG_Train_with_anomaly.csv', sep='\t', index_col=False, header=None)
test = pd.read_csv('C:/Users/Pro/Desktop/AnomalyDiT-main/AnomalyDiT-main/Dataset/ECG_Test_with_anomaly.csv', sep='\t', index_col=None, header=None)

In [6]:
train = torch.Tensor(train.drop(columns=0).to_numpy()).unsqueeze(1)
test = torch.Tensor(test.drop(columns=0).to_numpy()).unsqueeze(1)

In [7]:
test.shape

torch.Size([3930, 1, 0])

In [8]:
net = DiT(input_size=750,
        patch_size=5,
        in_channels=1,
        hidden_size=300,
        depth=28,
        num_heads=10,
        mlp_ratio=4.0,
        class_dropout_prob=0.1,
        learn_sigma=False,)

In [9]:
x = torch.randn(4,1,750)
y = test[:4]
t = torch.randint(low=0, high=10000, size=(4,))